# Homework 3: Word Embeddings
In this homework, we will try to approximate a Skip-gram word embedding via positive pointwise mutual information (PPMI) and truncated singular value decomposition (SVD). 

## The setup
Let's import the required libraries and load the data for preparing our word vectors. We are going to load a list of movie plot summaries (http://www.cs.cmu.edu/~ark/personas/) and use that as our corpus. You do not need to modify them.

In [0]:
# This code gets the data file from github and imports them into Colab
%%capture
!wget https://raw.githubusercontent.com/dbamman/nlp20/master/HW_3/plot_summaries_tokenized.csv

In [48]:
import numpy as np
import pandas as pd
from string import punctuation
from collections import Counter, defaultdict
from math import log2
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import svds

def load_data():
    """
    Loads the data and returns tokenized summaries.
    
    :return summaries_tokenized: a list that contains tokenized summaries text
    """
    df = pd.read_csv("plot_summaries_tokenized.csv")
    summaries_tokenized = list(df['SUMMARY'].apply(lambda text: text.split()))
    return summaries_tokenized

summaries = load_data()
num_summaries = len(summaries)
print("There are {} summaries.".format(num_summaries))
print("Example tokenized summary:", summaries[0])

There are 42303 summaries.
Example tokenized summary: ['Shlykov', 'a', 'hardworking', 'taxi', 'driver', 'and', 'Lyosha', 'a', 'saxophonist', 'develop', 'a', 'bizarre', 'lovehate', 'relationship', 'and', 'despite', 'their', 'prejudices', 'realize', 'they', 'arent', 'so', 'different', 'after', 'all']


We have ~42000 summaries containing ~13000000 words. We will now proceed by creating a vocabulary and will limit its size to something computationally feasible. You may find python's collections.Counter function useful. You may not import any additional libraries.

# 1. Create Vocabulary
We will start from creating our vocabulary. Vocabulary contains unigrams and their counts.

In [0]:
###################
# Do not modify
###################
min_count = (1 / 100) * len(summaries)
max_count = (1 / 10) * len(summaries)

In [0]:
def creat_vocabulary(tokenized_documents, min_count, max_count):
  """
  This function takes in tokenized documents and returns a
  vocabulary and word <-> index lookup dictionary of some frequently appearing words.
  
  :param tokenized_documents: a list of tokenized strings
  :param min_count: minimum unigram count
  :param max_count: maximum unigram count
  :return vocab: a Counter where vocab[word] = count of word's occurences in all the documents
  :return word2idx: a word -> index lookup Dictionary for words in vocab.
  :return idx2word: a index -> word lookup Dictionary for words in vocab.
  """
  # 1a. Compute unigram counts. A unigram is a single word, e.g. foo
  vocab = Counter()
  ##################################
  for summary in tokenized_documents:
    for word in summary:
      vocab[word] += 1
  ##################################
    
  # 1b. Remove unigrams that has #(unigram) < min_count or #(unigram) > max_count
  # to eliminate unigrams occurring very frequently or infrequently. 
  # This will limit its size to something computationally feasible.
  print('%d vocabs before' % len(vocab))
  ##################################
  for summary in tokenized_documents:
    for word in summary:
      if vocab[word] < min_count or vocab[word] > max_count:
        del vocab[word]
  ##################################
  print('%d vocabs after' % len(vocab))
        
  # 1c. Build word <-> index lookup for words in vocab.
  word2idx, idx2word = {}, {}
  ##################################
  i = 0
  while i < len(vocab):
    for word in vocab.keys():
      word2idx[word]=i
      idx2word[i]=word
      i += 1
  ##################################
  
  return vocab, word2idx, idx2word

In [51]:
vocab, word2idx, idx2word = creat_vocabulary(summaries, min_count, max_count)

214147 vocabs before
2750 vocabs after


# 2. Build Term-Context Matrix

In [0]:
###################
# Do not modify
###################
window_size = 3

In [0]:
def build_term_context_matrix(tokenized_documents, vocab, window_size):
  """
  This function returns a `word_pair_count` Counter with each 
  word_pair_count[(w, c)] = number of times the word `c` occurs in the context of word `w`. (where `w`, `c` belong to the vocab)
  To make it efficient, instead of building the sparse term-context matrix, 
  we will build 3 separate Counters: word_pair_count, w_count, c_count
  You may find python's Counter useful here

  :param tokenized_documents: a list of tokenized strings
  :param vocab: vocabulary Counter
  :param window_size: context window size
  :return word_pair_count: a Counter where word_pair_count[(w, c)] = count of c's occurences in w's context window, i.e. #(w, c)
  :return w_count: a Counter where w_count[w] = the number of times w occured in the documents, i.e. #(w)
  :return c_count: a Counter where c_count[c] = the number of times c occured in the documents, i.e. #(c)
  """
  word_pair_count = Counter()  
  w_count = Counter()
  c_count = Counter()
  ##################################
  for summary in tokenized_documents:
    for i in np.arange(len(summary)):
      if summary[i] in vocab.keys():
        target_index = i
        window_start = target_index-window_size
        window_end = target_index+window_size
        if (window_start < 0):
          window_start = 0
        if (window_end > len(summary)-1):
          window_end = len(summary)-1
        for j in np.arange(window_start, window_end+1):
          if j != target_index and summary[j] in vocab.keys():
            word_pair_count[(summary[i], summary[j])] += 1
            w_count[summary[i]]+=1
            c_count[summary[j]]+=1
  """
  for word in vocab.keys():
    for key in word_pair_count.keys():
      if (key[0] == word):
        w_count[word] +=1
  
  for key in word_pair_count.keys():
    w_count[key[0]] +=1

  for key in word_pair_count.keys():
    c_count[key[1]] +=1
  """
  ##################################
  
  return word_pair_count, w_count, c_count

In [56]:
word_pair_count, w_count, c_count = build_term_context_matrix(summaries, vocab, window_size)
print("There are {} word-context pairs".format(len(word_pair_count)))

# The number of w_count and c_count should match your number of vocab
assert len(w_count) == len(vocab)
assert len(c_count) == len(vocab)

There are 1917545 word-context pairs


# 3. Build Positive Pointwise Mutual Information (PPMI) Matrix
In this part, you will build a PPMI matrix using Scipy's Compressed Sparse Column matrix to save storage space. (https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csc_matrix.html)

Sparse matrix is a matrix which contains very few non-zero elements. When a sparse matrix is represented with a 2-dimensional array, we waste a lot of space to represent that matrix. In NLP application, it's quite common to use sparse matrix since the size of vocabulary is usually very large. 

Below is an example of how to build a sparse matrix where `data`, `row` and `col` satisfy the relationship `M[row[k], col[k]] = data[k]`.

```python
>>> row = np.array([0, 2, 2, 0, 1, 2])
>>> col = np.array([0, 0, 1, 2, 2, 2])
>>> data = np.array([1, 2, 3, 4, 5, 6])
>>> M = csc_matrix((data, (row, col)))
>>> M.toarray()
array([[1, 0, 4],
       [0, 0, 5],
       [2, 3, 6]])
```

Recall that
$$
\begin{gather*}
  \text{PMI}(w, c) = \log_2 \frac{P(w, c)}{P(w)P(c)} \\
  \text{PPMI}(w, c) = \max(0, \text{PMI}(w, c))
\end{gather*}
$$
You should use `log2` function from the math package that is alreadly imported for you.

In [0]:
def build_PPMI_matrix(word_pair_count, word_count, c_count, word2idx):
  """
  This function returns a PPMI matrix represented by a csc sparse matrix.

  :params word_pair_count: a Counter where word_pair_count[(w, c)] = count of c's occurences in w's context window
  :return w_count: a Counter where w_count[w] = the number of times w occured in the documents
  :return c_count: a Counter where c_count[c] = the number of times c occured in the documents
  :return word2idx: a word -> index lookup Dictionary for words in vocab
  :return PPMI: PPMI csc sparse matrix
  """
  data, rows, cols = [], [], []
  total_occurences = sum(word_pair_count.values())
  w_occurences = sum(w_count.values())
  c_occurences = sum(c_count.values())
  for (w, c), n in word_pair_count.items():
    ##################################
    p_w = w_count[w]/w_occurences
    p_c = c_count[c]/c_occurences
    PMI = np.math.log2((n/total_occurences)/(p_w*p_c))
    data.append(max(0,PMI))
    rows.append(word2idx[w])
    cols.append(word2idx[c])
    ##################################
  PPMI = csc_matrix((data, (rows, cols)))
  return PPMI

In [74]:
PPMI = build_PPMI_matrix(word_pair_count, w_count, c_count, word2idx)
print(PPMI[word2idx['doctor'], word2idx['nurse']])
# The shape of PPMI matrix should match your number of vocab
assert PPMI.shape == (len(vocab), len(vocab))

4.278442269389912


# 4. Truncated SVD
In this part, we will obtain a dense low-dimensional vectors via truncated (rank-k) SVD. You should use `svds` function from Sicpy that is already imported for you to obtain the SVD factorization.
(https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.svds.html)


In [0]:
###################
# Do not modify
###################
rank = 20

In [0]:
def get_embeddings(PPMI, rank):
  """
  Reutrns the left singular vectors as word embeddings via truncated SVD

  :params PPMI: PPMI csc sparse matrix
  :params rank: number of singular values and vectors to compute
  :return u: left sigular vectors from sprase SVD
  :return s: the singular values from sparse SVD
  """
  ##################################
  u, s, vt = svds(PPMI, k=rank)
  ##################################

  return u, s

In [0]:
embeddings, _ = get_embeddings(PPMI, rank)
embeddings /= np.linalg.norm(embeddings, axis=1, keepdims=True)  # Normalize embeddings matrix

# The shape of the embeddings matrix should be (# vocab, rank)
assert embeddings.shape == (len(vocab), rank)

# Make sure embeddings is normalized
assert True == np.isclose(np.linalg.norm(embeddings[0]), 1)

# 5. Evaluate Word Embeddings via Cosine Similarity

Using cosine similarity as a measure of distance [§6.4 Jurafsky & Martin](https://web.stanford.edu/~jurafsky/slp3/6.pdf), we will now find the closest words to a certain word. We define cosine similarity as, $$cosine(\overrightarrow{v},\overrightarrow{w}) = \frac{\overrightarrow{v} \cdot \overrightarrow{w}}{\vert v \vert \vert w \vert}$$

Please complete the function below that calculates the 'K' closest words from the vocabulary. You may not use any additional libraries.

In [0]:
###################
# Do not modify
###################
num_neighbors = 5

In [0]:
def cosine_distances(matrix, vector):
  """
  The function takes in a matrix and a vector (both normalized) 
  and returns the cosine distances for this vector against all others.
  The pretrained embeddings are normalized.

  :params matrix: word embeddings matrix
  :params vector: word vector for a particular word
  :return distances: a cosine distances vector
  """
  ##################################
  distances = []
  for v in matrix:
    distances.append(np.dot(vector,v)/(np.linalg.norm(vector)*np.linalg.norm(v)))
  ##################################
  
  return  distances


def nearest_neighbors(embeddings, word, k, word2idx, idx2word):
  """
  For each query word, this function returns the k closest words from the vocabulary.

  :params embeddings: word embedding matrix
  :params word: query word
  :params k: number of cloest words to return
  :params word2idx: a word -> index lookup dictionary
  :params idx2word: a index -> word lookup dictionary
  :return nearest_neighbors: a list of cloest words
  """
  vector = embeddings[word2idx[word]]
  distances = cosine_distances(embeddings, vector)
  nearest_neighbors = []
  ##################################
  idx = np.asarray(distances).argsort()[-(k+1):][::-1]
  for i in idx:
    if i != word2idx[word]:
      nearest_neighbors.append(idx2word[i])
  ##################################
  
  return nearest_neighbors

In [88]:
query_words = ["doctor", "zombie", "robot", "eat", "bus", 'soldier', 'wound', 'agent', 'telephone']
for word in query_words:
    print(word, nearest_neighbors(embeddings, word, num_neighbors, word2idx, idx2word))

doctor ['Sophie', 'priest', 'doctors', 'Christine', 'Anna']
zombie ['infected', 'zombies', 'vampires', 'creatures', 'vampire']
robot ['alien', 'creature', 'weapon', 'machine', 'demon']
eat ['throw', 'sleep', 'wear', 'stand', 'sit']
bus ['road', 'boat', 'truck', 'airport', 'river']
soldier ['Army', 'captain', 'Korean', 'Japanese', 'German']
wound ['wounds', 'arm', 'leg', 'neck', 'dying']
agent ['CIA', 'spy', 'executive', 'undercover', 'Lt']
telephone ['phone', 'paper', 'newspaper', 'camera', 'note']


# 6. Evaluate Word Embeddings via Analogous Tasks

The embedding space is known to capture the semantic context of words. An example of it is $\overrightarrow{woman} - \overrightarrow{man} \simeq \overrightarrow{queen} - \overrightarrow{king}$. Use the `cosine_distances()` function you wrote above to find such relations.

In [0]:
def relation(embeddings, query_words, word2idx, idx2word):
  """
  Takes in 3 words and returns the closest word (in terms of cosine similarity)
  to the normalized algebraic addition of the three vectors.
  The parameters follow this order : word_vec1 - word_vec2 ~ closest - word_vec3

  :params embeddings: word embedding matrix
  :params query_words: a list of query words in the following order: [word1, word2, word3]
  :params word2idx: a word -> index lookup dictionary
  :params idx2word: a index -> word lookup dictionary
  :return closet_word: the closest word for the relation
  """
  word1, word2, word3 = query_words
  if all(word in vocab for word in query_words):
    ##################################
    word1v = embeddings[word2idx[word1]]
    word2v = embeddings[word2idx[word2]]
    word3v = embeddings[word2idx[word3]]
    query = word1v - word2v + word3v
    query /= np.linalg.norm(query)
    embeddings = np.delete(embeddings, word2idx[word3], 0)
    distances = cosine_distances(embeddings, query)
    #del distances[word2idx[word3]]
    idx = np.argmax(distances)
    closet_word = idx2word[idx]
    ##################################
    
    return closet_word
  else:
    missing = [w for w in query_words if w not in vocab]
    raise Exception("missing {} from vocabulary".format(", ".join(missing)))

In [90]:
queries = [["doctor", "nurse", "king"], ["robot", "weapon", "bus"], ["sing", "song", "justice"], ["elderly", "kids", "teenager"], ["soldier", "wound", "telephone"]]
for query in queries:
  closet_word = relation(embeddings, query, word2idx, idx2word)
  print("{} - {} ~= {} - {}".format(query[0], query[1], closet_word, query[2]))

doctor - nurse ~= Emperor - king
robot - weapon ~= road - bus
sing - song ~= defend - justice
elderly - kids ~= widow - teenager
soldier - wound ~= agent - telephone
